#Recommendation Product
###by Snehasish Pradhan

##Python libraries

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import collections

In [0]:
import nltk

In [0]:
from nltk.corpus import stopwords

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
pd.options.display.max_columns = None

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

In [0]:
data = drive.CreateFile({'id': '1niIWXy1-gO3i-J3dYT-JC8xAJmnmd-ZC'})
data.GetContentFile('Reviews.csv.zip')

In [0]:
dataset = pd.read_csv('Reviews.csv.zip')

##check for null Values

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
Id                        568454 non-null int64
ProductId                 568454 non-null object
UserId                    568454 non-null object
ProfileName               568438 non-null object
HelpfulnessNumerator      568454 non-null int64
HelpfulnessDenominator    568454 non-null int64
Score                     568454 non-null int64
Time                      568454 non-null int64
Summary                   568427 non-null object
Text                      568454 non-null object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [7]:
' ' in dataset.Text

False

In [8]:
'' in dataset.Text

False

In [0]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
dataset.Time = pd.to_datetime(dataset.Time,unit='s')

In [0]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
dataset['score_sentiment'] = dataset['Score'].apply(lambda x: 1 if x>=4 else 0)

##Score sentiments

In [0]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score_sentiment
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""Delight"" says it all",This is a confection that has been around a fe...,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,Cough Medicine,If you are looking for the secret ingredient i...,0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,Great taffy,Great taffy at a great price. There was a wid...,1


In [0]:
dataset.HelpfulnessDenominator.max(),dataset.HelpfulnessDenominator.min()

(923, 0)

In [0]:
dataset.HelpfulnessNumerator.max(),dataset.HelpfulnessNumerator.min()

(866, 0)

##Converting HelpfulnessNumerator and HelpfulnessDenominator into average likes

In [0]:
dataset['avg_likes'] =      dataset.HelpfulnessNumerator / dataset.HelpfulnessDenominator

In [0]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score_sentiment,avg_likes
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,1.0
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,NaN
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""Delight"" says it all",This is a confection that has been around a fe...,1,1.0
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,Cough Medicine,If you are looking for the secret ingredient i...,0,1.0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,Great taffy,Great taffy at a great price. There was a wid...,1,NaN


##Removing nan values in average likes

In [0]:
dataset.avg_likes.fillna(0,inplace=True)

In [0]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score_sentiment,avg_likes
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,1.0
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,0.0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""Delight"" says it all",This is a confection that has been around a fe...,1,1.0
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,Cough Medicine,If you are looking for the secret ingredient i...,0,1.0
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,Great taffy,Great taffy at a great price. There was a wid...,1,0.0


In [0]:
dataset.avg_likes.max()

3.0

In [0]:
dataset.avg_likes.min()

0.0

In [0]:
dataset[dataset.avg_likes==3.0]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score_sentiment,avg_likes
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,2008-10-25,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...,1,3.0


##Sentiment for average likes on the basis of thresold value 
###avg_likes is our target value

In [0]:
dataset['avg_likes'] = dataset['avg_likes'].apply(lambda x: 1 if x>0.5 else 0)

In [0]:
dataset.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,score_sentiment,avg_likes
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,1
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,0
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""Delight"" says it all",This is a confection that has been around a fe...,1,1
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,Cough Medicine,If you are looking for the secret ingredient i...,0,1
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,Great taffy,Great taffy at a great price. There was a wid...,1,0


##Required Data

In [0]:
req_data = dataset[['ProductId','Text','avg_likes']].copy()

## converting to lower case

In [0]:
req_data['Text'] = req_data['Text'].apply(lambda x: x.lower())

In [0]:
req_data.head()

,ProductId,Text,avg_likes
0,B001E4KFG0,i have bought several of the vitality canned d...,1
1,B00813GRG4,product arrived labeled as jumbo salted peanut...,0
2,B000LQOCH0,this is a confection that has been around a fe...,1
3,B000UA0QIQ,if you are looking for the secret ingredient i...,1
4,B006K2ZZ7K,great taffy at a great price. there was a wid...,0


In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

##Removing Stopwords

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
req_data['Text'] = req_data['Text'].apply(lambda x : ' '.join(term for term in x.split() if term not in stop_words))

In [0]:
req_data.head()

,ProductId,Text,avg_likes
0,B001E4KFG0,bought several vitality canned dog food produc...,1
1,B00813GRG4,product arrived labeled jumbo salted peanuts.....,0
2,B000LQOCH0,"confection around centuries. light, pillowy ci...",1
3,B000UA0QIQ,looking secret ingredient robitussin believe f...,1
4,B006K2ZZ7K,great taffy great price. wide assortment yummy...,0


##Tokenize the required Data Text

In [0]:
num_words = 20000

In [0]:
tokenizer = Tokenizer(num_words = num_words)

In [0]:
%%time
tokenizer.fit_on_texts(req_data['Text'])

CPU times: user 33.3 s, sys: 74.4 ms, total: 33.3 s
Wall time: 33.4 s


In [0]:
req_data['Text'] = tokenizer.texts_to_sequences(req_data['Text'])

In [0]:
req_data.head()

,ProductId,Text,avg_likes
0,B001E4KFG0,"[54, 233, 4913, 439, 37, 12, 124, 47, 3, 106, ...",1
1,B00813GRG4,"[6, 287, 2160, 5742, 1911, 1020, 78, 1020, 167...",0
2,B000LQOCH0,"[7051, 191, 9228, 271, 1964, 3037, 448, 263, 1...",1
3,B000UA0QIQ,"[166, 2582, 497, 436, 47, 9, 74, 762, 1379, 13...",1
4,B006K2ZZ7K,"[7, 3531, 7, 27, 2025, 2091, 465, 3531, 583, 3...",0


In [0]:
num_tokens = [len(token) for token in req_data['Text']]
num_tokens = np.array(num_tokens)

In [0]:
num_tokens

array([24, 19, 37, ..., 39, 18,  9])

In [0]:
max_tokens = max(num_tokens)
max_tokens

2040

In [0]:
max_tokens = np.mean(num_tokens) + 2* np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

135

##Train and Test split

In [0]:
train, test, train_lbl, test_lbl = train_test_split(req_data['Text'],req_data['avg_likes'],test_size=0.2,random_state=1)

In [0]:
train.shape,train_lbl.shape,test.shape,test_lbl.shape

((454763,), (454763,), (113691,), (113691,))

##Padding the Tokenized Text

In [0]:
pad = 'pre'

In [0]:
train_pad = pad_sequences(train,maxlen = max_tokens,padding = 'pre',truncating = 'pre')

In [0]:
test_pad = pad_sequences(test,maxlen = max_tokens, padding = 'pre', truncating = 'pre')

In [0]:
train_pad.shape,test_pad.shape

((454763, 135), (113691, 135))

##Train and Validation split

In [0]:
pad_train, pad_val, pad_train_lbl, pad_val_lbl = train_test_split(train_pad,train_lbl,test_size=0.2,random_state=1)

In [0]:
pad_train.shape, pad_val.shape, pad_train_lbl.shape, pad_val_lbl.shape

((363810, 135), (90953, 135), (363810,), (90953,))

##LSTM Model

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, SpatialDropout1D

In [0]:
modell=Sequential()
modell.add(Embedding(num_words,32))
modell.add(LSTM(32))
modell.add(Dense(1, activation='sigmoid'))

In [0]:
modell.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])

In [0]:
modell.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          640000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 648,353
Trainable params: 648,353
Non-trainable params: 0
_________________________________________________________________


##Train and Validate the model

In [0]:
history = modell.fit(pad_train, pad_train_lbl, epochs=3, validation_data=(pad_val, pad_val_lbl), shuffle=False)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 363810 samples, validate on 90953 samples
Epoch 1/3
363810/363810 [==============================] - 1385s 4ms/step - loss: 0.6444 - acc: 0.6347 - val_loss: 0.6366 - val_acc: 0.6388
Epoch 2/3
363810/363810 [==============================] - 1408s 4ms/step - loss: 0.6226 - acc: 0.6586 - val_loss: 0.6276 - val_acc: 0.6534
Epoch 3/3
363810/363810 [==============================] - 1410s 4ms/step - loss: 0.5988 - acc: 0.6810 - val_loss: 0.6214 - val_acc: 0.6606


##Make prediction

In [0]:
q = modell.predict(test_pad)

##Prediction array

In [0]:
q

array([[0.21272124],
       [0.33921203],
       [0.13745414],
       ...,
       [0.12803917],
       [0.82945013],
       [0.6188991 ]], dtype=float32)

##Converting Prediction to Sentiment 0-1 on the basis of thresold value

In [0]:
q[q<0.45]=0
q[q>0.45]=1

In [0]:
q

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
test_lbl.shape,q.shape

((113691,), (113691, 1))

##Accuracy

In [0]:
accuracy_score(test_lbl, q)

0.6523735388025438

In [0]:
req_data1 = dataset[['ProductId','Text','avg_likes']].copy()

In [0]:
req_data1.head()

,ProductId,Text,avg_likes
0,B001E4KFG0,I have bought several of the Vitality canned d...,1
1,B00813GRG4,Product arrived labeled as Jumbo Salted Peanut...,0
2,B000LQOCH0,This is a confection that has been around a fe...,1
3,B000UA0QIQ,If you are looking for the secret ingredient i...,1
4,B006K2ZZ7K,Great taffy at a great price. There was a wid...,0


In [0]:
train1, prediction= train_test_split(req_data1[['ProductId','Text','avg_likes']],test_size=0.2,random_state=1)

In [0]:
prediction['predict_recommend'] = q

##Prediction

In [0]:
prediction

,ProductId,Text,avg_likes,predict_recommend
288312,B000ENUC3S,I love the Cherry Pie Lara bar. Best and tast...,0,0.0
431726,B002TMV3CG,Melitta Cafe COllection Blanc et Noir coffee h...,0,0.0
110311,B004867T24,my girls absolutely loved this tuna. they were...,0,0.0
91855,B004U7KPY0,The vendor is fast and dependable. The tea is ...,0,1.0
338855,B000FD78R0,UPDATE - 8/9/2010<br />A lot can happen in jus...,1,1.0
243608,B0059X9A6I,I have pretty much tried them all. The dark ch...,0,0.0
152343,B000LKZD4W,"Just be forewarned it is a bit ""moist."" Have ...",1,1.0
336202,B000BRR8VQ,Both of my mixed terrier dogs prefer this trea...,0,0.0
488611,B004JQVAVO,Made this coffee with a percolator maybe that ...,0,0.0
103618,B0083L5F8K,This was just the basic ingredients that i cou...,0,1.0
